In [ ]:
import os
import numpy as np
import scipy.sparse
import scipy.io
from sklearn.metrics.cluster import normalized_mutual_info_score as nmi
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from scipy import stats 
from scipy import * 
import numpy as np
import scanpy as sc
import pandas as pd

In [ ]:
dir_path = "/home/krushna/Documents/Data_integration/SCRNA_Datasets/All_h5ad/"
def load_data(dataset,batch):
    adata =sc.read_h5ad(dir_path+dataset+'.h5ad')
    # sc.pp.filter_genes(adata, min_counts=3)
    # adata.layers["counts"] = adata.X.copy()
    # sc.pp.normalize_total(adata, target_sum=1e4)
    # sc.pp.log1p(adata)
    # adata.raw = adata
    # sc.pp.highly_variable_genes(
    #         adata,
    #         flavor="seurat",
    #         n_top_genes=2000,
    #         layer="counts",
    #         batch_key=batch,
    #         subset=True
    # )
    # return adata
batch_key_dic = {'Immune_Human' : 'batch',
                 'Lung' : 'batch',
                 'Pancreas' : 'tech',
                 'Human_Mouse' : 'batch',
                 'Human_Retina': "Batch"
                 
                }
cell_type_key_dic = {'Immune_Human' : 'final_annotation',
                 'Lung' : 'cell_type',
                 'Pancreas' : 'celltype',
                 'Human_Mouse' : "celltype",
                 "Human_Retina":"Subcluster"
                    }    
our_method_emb_dic = {'Immune_Human' : 'Immune_Human_case2_latent_matrix_c_300',
                 'Lung' : 'Lung_case2_latent_matrix_c_300',
                 'Pancreas' : 'Pancreas_case2_latent_matrix_c_250',
                 'Human_Retina': "retina_case2_latent_matrix_c_300",
                 'Human_Mouse' : 'Human_Mouse_latent_matrix_c100'
                }   
emb_folder = "/home/krushna/Documents/Data_integration/Ajita_embeddings/misc/checking_other_cases/"

In [ ]:
dataset = 'Human_Mouse'
embeddings = our_method_emb_dic[dataset]+'.csv'
batch = batch_key_dic[dataset]
cell_type = cell_type_key_dic[dataset]

In [ ]:
adata = load_data(dataset,batch)
emb_path = emb_folder + embeddings 
final_embedings=pd.read_csv(emb_path, sep=',',header=None).values
adata.obsm['final_embeddings'] = final_embedings

In [ ]:
import scIB
adata = sc.read_h5ad(dir_path+dataset+'.h5ad')
embeddings = our_method_emb_dic[dataset]+'.csv'
emb_path = emb_folder + embeddings 
final_embedings=pd.read_csv(emb_path, sep=',',header=None).values
adata.obsm['final_embeddings'] = final_embedings
results,ilisi_all,clisi_all,kbet_all  =   scIB.metrics.metrics(                
                adata,
                adata,
                batch_key = batch, #need to change according
                label_key = cell_type, #need to change according
                hvg_score_=False,
                cluster_key='cluster',
                cluster_nmi=None,
                ari_=False,
                nmi_=False,
                nmi_method='arithmetic',
                nmi_dir=None,
                silhouette_=False,
                embed= 'final_embeddings', #need to change according 
                si_metric='euclidean',
                pcr_=True,
                cell_cycle_=False,
                organism='mouse',
                isolated_labels_=False,  # backwards compatibility
                isolated_labels_f1_=False,
                isolated_labels_asw_=False,
                n_isolated=None,
                graph_conn_=True,
                kBET_=False,
                kBET_sub=0.5,
                lisi_graph_=False,
                lisi_raw=False,
                trajectory_=False,
                type_=None,
                verbose=True,
        )
results

In [ ]:
import scIB
results,ilisi_all,clisi_all,kbet_all  =   scIB.metrics.metrics(                
                adata,
                adata,
                batch_key = batch, #need to change according
                label_key = cell_type, #need to change according
                hvg_score_=False,
                cluster_key='cluster',
                cluster_nmi=None,
                ari_=True,
                nmi_=True,
                nmi_method='arithmetic',
                nmi_dir=None,
                silhouette_=True,
                embed= 'final_embeddings', #need to change according 
                si_metric='euclidean',
                pcr_=True,
                cell_cycle_=False,
                organism='mouse',
                isolated_labels_=False,  # backwards compatibility
                isolated_labels_f1_=False,
                isolated_labels_asw_=False,
                n_isolated=None,
                graph_conn_=True,
                kBET_=True,
                kBET_sub=0.5,
                lisi_graph_=True,
                lisi_raw=True,
                trajectory_=False,
                type_=None,
                verbose=True,
        )

In [ ]:
results

In [ ]:
import numpy as np
path_to_save = "/home/krushna/Documents/Data_integration/ourmethod_metrics/kbet_lisi/"
np.savetxt(path_to_save+dataset+"_ilisi.csv", ilisi_all, delimiter=",")
np.savetxt(path_to_save+dataset+"_clisi.csv", clisi_all, delimiter=",")
np.savetxt(path_to_save+dataset+"_kbet_all.csv",np.concatenate([np.array(val).reshape(1,-1) for val in kbet_all],axis = 0), delimiter=',')

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import normalized_mutual_info_score as nmi


latent_matrix = adata.obsm['final_embeddings'] 
labels = np.array(adata.obs[cell_type] )
K = np.size(np.unique(labels))
kmeans = KMeans(n_clusters=K, random_state=0).fit(latent_matrix)
y_pred = kmeans.labels_

print('Computing NMI ...')
NMI = nmi(labels.flatten(), y_pred.flatten())
print('NMI = {}'.format(NMI))

In [ ]:
sc.pp.neighbors(adata, use_rep='final_embeddings')  # use_rep = 'final_embeddings'
sc.tl.umap(adata)
sc.pl.umap(adata, color=cell_type, frameon=False)
sc.pl.umap(adata, color=batch, frameon=False)
sc.pl.umap(adata, color='cluster', frameon=False)